# Raw file sizes

Author: Melissa

Explore the raw file sizes of the parquet files that back butler queries.

This isn't super-useful, but I'm waiting for some slow import steps and I'm having fun.



In [11]:
# Generic python packages
import pylab as plt
import gc
import numpy as np
import os

# LSST Science Pipelines (Stack) packages
import lsst.daf.butler as dafButler
import lsst.afw.display as afwDisplay
import pandas as pd
import pyarrow.parquet as pq
from pathlib import Path

# Set a standard figure size to use
plt.rcParams['figure.figsize'] = (8.0, 8.0)
afwDisplay.setDefaultBackend('matplotlib')

In [12]:

base_output_dir = Path("/sdf/data/rubin/shared/lsdb_commissioning/dm_48556")

In [13]:

def get_all_sizes(dataset_type):
    file_pointer = base_output_dir / "raw" /"paths"/ f"{dataset_type}.txt"
    
    with file_pointer.open("r", encoding="utf8") as _text_file:
        paths = _text_file.readlines()
    paths = [path.strip() for path in paths]

    ref_frame = pd.read_csv(base_output_dir / "raw"  / "refs" / f"{dataset_type}.csv") 
    ref_frame["paths"] = paths
    
    num_columns = []
    num_rows = []
    num_row_groups = []
    file_size = []
    
    for path in paths:
        parquet_md = pq.ParquetFile(path.strip()).metadata
        num_columns.append(parquet_md.num_columns)
        num_rows.append(parquet_md.num_rows)
        num_row_groups.append(parquet_md.num_row_groups)
        file_size.append(os.path.getsize(path))
    
    ref_frame["num_columns"] = num_columns
    ref_frame["num_rows"] = num_rows
    ref_frame["num_row_groups"] = num_row_groups
    ref_frame["file_size"] = file_size
    ref_frame["gbs"] = ref_frame["file_size"] / (1024 * 1024 * 1024)
    
    ref_frame.to_csv(base_output_dir / "raw" / "sizes" / f"{dataset_type}.csv", index=False)

In [14]:
dataset_types = ['diaObjectTable_tract', 'diaSourceTable_tract', 'forcedSourceOnDiaObjectTable',
                 'objectTable', 'sourceTable', 'forcedSourceTable']

In [15]:
# for set_type in dataset_types:
#     get_all_sizes(set_type)

## Estimate the pixel thresholds.

Using something similar to [this old notebook](https://hats-import.readthedocs.io/en/latest/notebooks/estimate_pixel_threshold.html), but using the full dataset size and row count, we can get a pretty good idea of what good pixel thresholds are for each dataset.

In [20]:
def print_import_stats(dataset_type):
    all_sizes = pd.read_csv(base_output_dir / "raw" / "sizes" / f"{dataset_type}.csv")
    sample_file_size=all_sizes["file_size"].sum()
    num_rows=all_sizes["num_rows"].sum()
    ## 300MB
    ideal_file_small = 300 * 1024 * 1024
    ## 1G
    ideal_file_large = 1024 * 1024 * 1024
    
    threshold_small = ideal_file_small / sample_file_size * num_rows
    threshold_large = ideal_file_large / sample_file_size * num_rows
    print(dataset_type)
    print(f"  threshold between {int(threshold_small):_} and {int(threshold_large):_}")
    print(f'  total size_on_disk: {all_sizes["gbs"].sum():.2f} G')

In [21]:
for set_type in dataset_types:
    print_import_stats(set_type)

diaObjectTable_tract
  threshold between 1_741_018 and 5_942_676
  total size_on_disk: 0.68 G
diaSourceTable_tract
  threshold between 1_506_282 and 5_141_445
  total size_on_disk: 1.35 G
forcedSourceOnDiaObjectTable
  threshold between 3_454_977 and 11_792_991
  total size_on_disk: 90.32 G
objectTable
  threshold between 93_254 and 318_308
  total size_on_disk: 17.12 G
sourceTable
  threshold between 477_739 and 1_630_684
  total size_on_disk: 33.16 G
forcedSourceTable
  threshold between 3_443_957 and 11_755_373
  total size_on_disk: 50.36 G
